## Meals Count Backend Utilities  
  
This notebook contains Python functionality to be incorporated into modules part of the [MealsCount](https://github.com/opensandiego/mealscount-backend) backend. **MealsCount** is an open-source project currently underway at [**OpenSanDiego**](https://opensandiego.org/), a [Code for America](https://www.codeforamerica.org/) brigade.  
  
**Note**: See [**[1]**](backend_utils_test.ipynb) for a detailed explanation of the core functionality, including intermediate outputs based on sample test data.   

In [1]:
import sys
import os
import pandas as pd
import numpy as np

from abc import ABC, abstractmethod

In [2]:
#
# GLOBAL CONSTANTS (DO NOT MODIFY)
#

# these are used to identify data rows 
# level 1 header
DATA_L1_HDR_KEYS = ['Non-Charter School(s)','Charter School(s)']
# level 2 header
DATA_L2_HDR_KEYS = ['School Code','School Name','Total Enrollment','Free & Reduced Meal Program: 181/182',
                    'Foster','Homeless(1)','Migrant Program: 135','Direct Certification',
                    'Unduplicated Eligible Free/Reduced Meal Counts','EL Funding Eligible (2)',
                    'Total Unduplicated FRPM/EL Eligible (3)']
# keywords for aggregated rows
DATA_SUM1_KEYS = ['TOTAL - Selected Schools']
DATA_SUM2_KEYS = ['TOTAL LEA']

# these are used for recoding header names/col values where applicable
DATA_L1_HDR_DICT = {'Non-Charter School(s)':'non-charter','Charter School(s)':'charter'}
DATA_L2_HDR_DICT = {'School Code':'school_code','School Name':'school_name','Total Enrollment':'total_enrolled',
                    'Free & Reduced Meal Program: 181/182':'frpm','Foster':'foster','Homeless (1)':'homeless',
                    'Migrant Program: 135':'migrant','Direct Certification':'direct_cert',
                    'Unduplicated Eligible Free/Reduced Meal Counts':'frpm_nodup',
                    'EL Funding Eligible (2)':'el', 'Total Unduplicated FRPM/EL Eligible (3)':'frpm_el_nodup'}
DATA_SUM_DICT = {'TOTAL - Selected Schools':'total','TOTAL LEA':'total'}

# these are used for recoding specific col values
INVALID_SCHOOL_CODE = 9999999
ALL_SCHOOL_TYPE = 'lea'

# these are used to identify metadata rows
METADATA_KEYS = ['Academic Year','View','As Of','Gender','School Type','School','User ID',
                 'Created Date','LEA']
# these are used to identify cols corresponding to 
# metadata key-value pairs
METADATA_KEY_COLS = [0,2,4]
METADATA_VAL_COLS = [1,3,5]

In [3]:
class mcSchoolDistInput(ABC):
    """
    Base class for school district input. 
    """
    d_df = pd.DataFrame()
    md_dict = {}
    
    def __init__(self):                
        pass
    
    @abstractmethod
    def to_frame(self):
        pass
    
    @abstractmethod
    def metadata(self):
        pass

In [4]:
#
# Function to extract and return a dataframe from the input
# dataframe and the row and col indices specified. Additionally
# a column for school type is added with the specified value as
# well as a column (called 'index') with the original row indices.
#
def extract_df(df,row_idx,col_idx,school_type):
    
    data = df.loc[row_idx,:].values
    cols = df.loc[col_idx].values
    ext_df = pd.DataFrame(data=data,columns=cols)        
    
    ext_df['school_type'] = school_type 
    ext_df['index'] = row_idx
    
    ext_df.dropna(axis=1,how='all',inplace=True)
    ext_df.dropna(axis=0,how='all',inplace=True)
    
    return ext_df

In [5]:
# FIXME: refactor code in here
def parseXL(self,xlfile):
    
    try:
        xl = pd.ExcelFile(xlfile)                
        tmpdf = xl.parse(xl.sheet_names[0])       
        
        # get the indices for the rows where the L1 headers are present
        data_l1 = tmpdf.index[tmpdf[tmpdf.isin(DATA_L1_HDR_KEYS)].notnull().any(axis=1)].tolist()
        # get indices for rows where the L2 headers are present
        # these will indicate the beginning of data 
        data_l2_begin = tmpdf.index[tmpdf[tmpdf.isin(DATA_L2_HDR_KEYS)].notnull().any(axis=1)].tolist()
        # get indices for the rows where the misc headers are present
        # these will indicate the end of data
        data_l2_end = tmpdf.index[tmpdf[tmpdf.isin(DATA_SUM1_KEYS)].notnull().any(axis=1)].tolist()
        # get indices for any other keys that are part of data
        data_other = tmpdf.index[tmpdf[tmpdf.isin(DATA_SUM2_KEYS)].notnull().any(axis=1)].tolist()

        # generate indices of non-data rows 
        metadata_idx = list(range(0,data_l1[0]))
        n = len(DATA_L1_HDR_KEYS)

        # TODO: malformed files may have any of the keys missing resulting in 
        # empty lists of indices

        for i in range(0,n):    
            metadata_idx += list(range(data_l1[i]+1,data_l2_begin[i]))
            if i < n-1:
                metadata_idx += list(range(data_l2_end[i]+1,data_l1[i+1]))

        metadata_idx += list(range(data_l2_end[n-1]+1,data_other[0]))
        metadata_idx += list(range(data_other[-1]+1,tmpdf.shape[0]))
        
        # copy metadata rows to its own dataframe
        tmpdf_md = tmpdf.loc[metadata_idx,:]
        # clean-up
        tmpdf_md.dropna(axis=1,how='all',inplace=True)
        tmpdf_md.dropna(axis=0,how='all',inplace=True)

        # purge metadata rows (copied above) from the data df
        tmpdf.drop(metadata_idx,inplace=True)
        # clean-up
        tmpdf.dropna(axis=1,how='all',inplace=True)
        tmpdf.dropna(axis=0,how='all',inplace=True)
        
        # collect l1 header names
        # needed because we don't know the order in which the l1 headers occur in data
        df_l1 = tmpdf.loc[data_l1]
        df_l1 = df_l1.loc[:,df_l1.notnull().any()]
        l1_hdrs = df_l1.T.unstack().tolist()
        l1_hdrs = [s for s in l1_hdrs if str(s) != 'nan']
        
        # drop all l1 headers
        # we will be using a single-level index for the final df
        # l1 headers will be used to populate a categorical var instead
        tmpdf.drop(data_l1,inplace=True)

        # create a new ddtaframe for each school type
        df_list = []
        for i in range(0,n):
    
            row_idx = list(range(data_l2_begin[i]+1,data_l2_end[i]+1))   
            col_idx = data_l2_begin[i]
            school_type = l1_hdrs[i]    
    
            df_list.append(extract_df(tmpdf,row_idx,col_idx,school_type))
    
            # if this the last of the school types we need to append
            # the aggregated lea rows. we do this as a separate df containing
            # data_other rows.
            if (i==n-1):
                row_idx = data_other
                df_list.append(extract_df(tmpdf,row_idx,col_idx,np.nan))
        
        # we have a df with all data for all school types including aggregated
        # rows at this point
        df_full = pd.concat(df_list,axis=0,ignore_index=True)
        
        # recode column names
        df_full.rename(columns=DATA_L2_HDR_DICT,inplace=True)
        # recode school_type
        df_full['school_type'] = df_full['school_type'].map(DATA_L1_HDR_DICT)
        # recode other fields
        cond = df_full['index'].isin(data_l2_end + data_other)
        df_full.loc[cond,'school_name'] = df_full[cond]['school_code'].map(DATA_SUM_DICT)
        df_full.loc[cond,'school_code'] = INVALID_SCHOOL_CODE
        cond = df_full['index'].isin(data_other)
        df_full.loc[cond,'school_type'] = ALL_SCHOOL_TYPE
        
        df_full.drop(['index'],axis=1,inplace=True)
        # re-arrange cols to original order
        df_full = df_full[list(DATA_L2_HDR_DICT.values()) + ['school_type']]
        
        #
        # METADATA
        #
        
        # add appropriate prefix and suffix to metadata keys 
        md_keys = ['   ' + s + ':' for s in METADATA_KEYS]
        # get indices for rows where the metadata keywords are present
        md_idx = tmpdf_md.index[tmpdf_md[tmpdf_md.isin(md_keys)].notnull().any(axis=1)].tolist()
        
        # extract non-null cols only for those rows containing metadata keys
        tmpdf_md = tmpdf_md.loc[md_idx,:]
        tmpdf_md.dropna(axis=1,how='all',inplace=True)
        tmpdf_md.dropna(axis=0,how='all',inplace=True)
        tmpdf_md.columns = list(range(0,tmpdf_md.shape[1]))

        # extract metadata keys
        md_keys = list(tmpdf_md.loc[:,METADATA_KEY_COLS].unstack().values)
        md_keys = list(map(str.strip,md_keys))
        md_keys = list(map(str.lower,md_keys))
        md_keys = [s.replace(' ','_') for s in md_keys]
        md_keys = [s[:-1] for s in md_keys]
        
        # extract metadata values
        md_vals = list(tmpdf_md.loc[:,METADATA_VAL_COLS].unstack().values)
        md_vals = [s.lower() if isinstance(s, str) else s for s in md_vals]

        md_dict = dict(zip(md_keys, md_vals))
        
        # store only at the end when we have successfully completed all steps
        # for both data and metadata
        self.d_df = df_full
        self.md_dict = md_dict
                
    except Exception as e:
        raise e

In [6]:
class mcXLSchoolDistInput(mcSchoolDistInput):
    """
    Implementation for MealsCount Excel format school district input. Expects input 
    to be a file stored on the backend.
    """        
    
    def __init__(self, datafile):        
        mcSchoolDistInput.__init__(self)
        self.__datafile = datafile
        try:
            self.__parse(self.__datafile)
        except Exception as e:
            raise e
    
    def to_frame(self):
        return self.d_df
    
    def metadata(self):
        return self.md_dict
    
    __parse = parseXL           
        

### Usage    
  
Below code fragments demonstrate the usage of the above functionality.    

In [7]:
CWD = os.getcwd()

DATADIR = "data"
DATAFILE = "calpads_sample_data.xlsx"

In [8]:
data_in = mcXLSchoolDistInput(os.path.join(DATADIR,DATAFILE))
df = data_in.to_frame()
df.head()

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type
0,1000001,School NC01,37,4,27,0,0,6,29,5,30,non-charter
1,1000002,School NC02,1111,503,2,7,0,215,527,122,556,non-charter
2,1000003,School NC03,2332,897,2,14,0,440,979,169,1037,non-charter
3,1000004,School NC03,854,733,6,10,1,361,765,296,792,non-charter
4,1000005,School NC04,1628,1119,7,17,0,640,1239,474,1333,non-charter


In [9]:
df.tail()

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type
31,2000001,School C01,460,321,0,0,0,182,332,192,364,charter
32,2000002,School C02,420,313,0,2,0,191,323,253,363,charter
33,2000003,School C03,246,119,0,11,0,85,152,187,222,charter
34,9999999,total,1126,753,0,13,0,458,807,632,949,charter
35,9999999,total,40319,21800,111,528,19,12279,23704,8673,25663,lea


In [10]:
df[df['school_name']=='total']

,school_code,school_name,total_enrolled,frpm,foster,homeless,migrant,direct_cert,frpm_nodup,el,frpm_el_nodup,school_type
30,9999999,total,39193,21047,111,515,19,11821,22897,8041,24714,non-charter
34,9999999,total,1126,753,0,13,0,458,807,632,949,charter
35,9999999,total,40319,21800,111,528,19,12279,23704,8673,25663,lea


In [11]:
metadata = data_in.metadata()
print(metadata)

{'academic_year': '2017-2018', 'view': 'ods', 'as_of': datetime.datetime(2018, 3, 12, 0, 0), 'gender': 'all', 'school_type': 'all', 'school': 'all', 'user_id': nan, 'created_date': '03-12-2018', 'lea': 'somecity union high'}


### TODO  
  
* Refactor ParseXL functionality  
* Add API for error status ?
* Profile parser for large datasets ?